In [27]:
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import numpy as np

In [28]:
#check total number of sequeces retrieved
ids = []
for record in SeqIO.parse('wrky.fasta','fasta'):
    ids.append(record.id)
print(len(ids))

180


In [29]:
def is_valid_cds(cds):
    """Check if a sequence meets the six quality criteria."""
    return (cds.startswith("ATG") and cds.endswith(("TAA", "TAG", "TGA")) 
            and len(cds) % 3 == 0 and len(cds) >= 300 
            and "TAA" not in cds[3:-3] and "TAG" not in cds[3:-3] and "TGA" not in cds[3:-3]
            and all(nuc in "ATGC" for nuc in cds))

def filter_sequences(sequences):
    """Filter sequences that pass the quality check."""
    return [seq for seq in sequences if is_valid_cds(str(seq.seq))]

def detect_length_outliers(sequences):
    """Detect outliers in sequence length using IQR."""
    if len(sequences) == 0:
        return []
    
    lengths = np.array([len(seq) for seq in sequences])
    q1, q3 = np.percentile(lengths, [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return [seq for seq, length in zip(sequences, lengths) if length < lower_bound or length > upper_bound]

def write_fasta(sequences, filename):
    """Write valid sequences to a FASTA file."""
    SeqIO.write(sequences, filename, "fasta")

In [30]:
# Load sequences from a FASTA file
sequences = list(SeqIO.parse("wrky.fasta", "fasta"))

In [31]:
# Perform filtering and outlier detection
filtered_sequences = filter_sequences(sequences)

In [32]:
# Check if any valid sequences exist before detecting outliers
if len(filtered_sequences) > 0:
    outliers = detect_length_outliers(filtered_sequences)
else:
    outliers = []

In [33]:
# Sequences that failed the quality check
discarded_sequences_count = len(sequences) - len(filtered_sequences)

# Write valid sequences to a new FASTA file
write_fasta(filtered_sequences, "GmWRKY.fasta")

In [34]:
# Print summary
print(f"Total sequences processed: {len(sequences)}")
print(f"Valid sequences: {len(filtered_sequences)}")
print(f"Length outliers: {len(outliers)}")
print(f"Discarded sequences (due to failing quality checks): {discarded_sequences_count}")

Total sequences processed: 180
Valid sequences: 0
Length outliers: 0
Discarded sequences (due to failing quality checks): 180
